In [ ]:
# !pip install pyspark

In [9]:
import os
import string
from nltk.corpus import stopwords
import pandas as pd
# nltk.download('stopwords')
# os.getcwd()

In [10]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc
import os

In [4]:
os.chdir("../..")
base_dir = f"{os.getcwd()}/00_input/data"
bible_path = f"{base_dir}/bible-kjv.txt"
shakes_path = f"{base_dir}/shakespeare.txt"

In [62]:
spark = SparkSession.builder\
    .master("local") \
    .appName("Word Count") \
    .config("spark.sql.hive.metastore.warehouse.dir", f"{base_dir}/00_output/hive/warehouse" )\
    .enableHiveSupport() \
    .getOrCreate()

sc = spark.sparkContext

In [12]:
punc: str = string.punctuation + "\n\t0123456789"
stop_words: list = stopwords.words("english") + ["th"]

In [13]:
def get_df(path):
    rdd = sc.textFile("file://" + path)
    words = rdd.flatMap(\
        lambda line: line\
            .lower()
            .translate(str.maketrans(punc, " " * len(punc)))\
            .split())

    wc_rdd = words\
        .filter(lambda word: word not in stop_words)\
        .map(lambda word: (word, 1))\
        .reduceByKey(lambda a, b: a + b)

    return wc_rdd.toDF(["words", "count"])

In [14]:
bible_wc_df = get_df(bible_path)

In [15]:
bible_wc_df = bible_wc_df.withColumnRenamed("count", "bb_count")

In [16]:
bible_wc_df.createOrReplaceTempView("bible")

In [17]:
spark.sql("SELECT * FROM bible ORDER BY bb_count DESC limit 100").show()

+------+--------+
| words|bb_count|
+------+--------+
| shall|    9838|
|  unto|    8997|
|  lord|    7964|
|  thou|    5474|
|   thy|    4600|
|   god|    4472|
|  said|    3999|
|    ye|    3983|
|  thee|    3827|
|  upon|    2748|
|   man|    2735|
|israel|    2575|
|  king|    2543|
|   son|    2392|
|  hath|    2264|
|people|    2143|
|  came|    2093|
| house|    2024|
|  come|    1971|
|   one|    1969|
+------+--------+
only showing top 20 rows



In [18]:
ordered_bible = spark\
    .sql("""
        SELECT words, bb_count, 
            RANK() OVER (ORDER BY bb_count DESC) rank_no 
        FROM bible""")

In [19]:
short_bible = ordered_bible\
    .filter(ordered_bible.rank_no < 101)\
    .select("words", "bb_count")

In [20]:
short_bible.show()

+------+--------+
| words|bb_count|
+------+--------+
| shall|    9838|
|  unto|    8997|
|  lord|    7964|
|  thou|    5474|
|   thy|    4600|
|   god|    4472|
|  said|    3999|
|    ye|    3983|
|  thee|    3827|
|  upon|    2748|
|   man|    2735|
|israel|    2575|
|  king|    2543|
|   son|    2392|
|  hath|    2264|
|people|    2143|
|  came|    2093|
| house|    2024|
|  come|    1971|
|   one|    1969|
+------+--------+
only showing top 20 rows



In [22]:
shake_wc_df = get_df(shakes_path)

In [23]:
shake_wc_df = shake_wc_df.withColumnRenamed("count", "sh_count")

In [26]:
short_shake_df = shake_wc_df.orderBy(desc("sh_count")).limit(100)

In [27]:
short_shake_df.count()

100

In [31]:
short_inner_df = short_shake_df.join(short_bible, on="words") 

In [34]:
short_inner_df.show(4)

+-----+--------+--------+
|words|sh_count|bb_count|
+-----+--------+--------+
|shall|    3600|    9838|
| lord|    3094|    7964|
| thou|    5549|    5474|
|  thy|    4034|    4600|
+-----+--------+--------+
only showing top 4 rows



In [41]:
short_outer_df = short_shake_df\
    .join(short_bible, on="words", how="outer")\
    .fillna(0)

In [42]:
short_outer_df.count()

157

In [43]:
short_outer_df.show(5)

+---------+--------+--------+
|    words|sh_count|bb_count|
+---------+--------+--------+
|according|       0|     797|
|     also|       0|    1769|
|    among|       0|     916|
|      art|     936|       0|
|     away|     862|     915|
+---------+--------+--------+
only showing top 5 rows



In [52]:
wc_df = shake_wc_df.join(bible_wc_df, on=["words"])

In [53]:
wc_df.show(5)

+---------+--------+--------+
|    words|sh_count|bb_count|
+---------+--------+--------+
|arguments|      11|       1|
|      art|     936|     494|
|    besom|       1|       1|
|  blossom|      10|       6|
|   brands|       8|       1|
+---------+--------+--------+
only showing top 5 rows



In [55]:
wc_df.orderBy(desc("sh_count")).show(5)

+-----+--------+--------+
|words|sh_count|bb_count|
+-----+--------+--------+
| thou|    5549|    5474|
|  thy|    4034|    4600|
|shall|    3600|    9838|
| thee|    3181|    3827|
| lord|    3094|    7964|
+-----+--------+--------+
only showing top 5 rows



In [47]:
spark.sql("""
    CREATE TABLE IF NOT EXISTS complete (
        words string,
        sh_count long,
        bb_count long
    )
""")

DataFrame[]

In [44]:
short_outer_df.printSchema()

root
 |-- words: string (nullable = true)
 |-- sh_count: long (nullable = true)
 |-- bb_count: long (nullable = true)



In [61]:
short_outer_df.write.mode("append").insertInto("complete")

In [60]:
spark.sql("select * from complete LIMIT 10").show() # write to hive table

+---------+--------+--------+
|    words|sh_count|bb_count|
+---------+--------+--------+
|according|       0|     797|
|     also|       0|    1769|
|    among|       0|     916|
|      art|     936|       0|
|     away|     862|     915|
|       ay|     769|       0|
|   behold|       0|    1326|
|    blood|     667|       0|
| brethren|       0|     564|
|    bring|       0|     725|
+---------+--------+--------+

